In [ ]:
import numpy as np
import cv2
import os
import pandas as pd
import time
import tensorflow as tf
from collections import deque

In [ ]:
# Define the path to your video and your converted TFLite model
V_DIR = '/home/kronbii/dataset/Violence'
NV_DIR = '/home/kronbii/dataset/NonViolence'
V_OUTPUT_DIR = '/home/kronbii/repos/content-violence-detection/output/test/V'
NV_OUTPUT_DIR = '/home/kronbii/repos/content-violence-detection/output/test/NV'
TFLITE_MODEL_PATH = '/home/kronbii/repos/violence-detection/converted_model.tflite'
CONFIDENCE = 0.400

In [ ]:
def process_dir(input_dir, base_output_dir):
    # --- 1. Load the TFLite model only ONCE for efficiency ---
    try:
        print("Loading TFLite model...")
        interpreter = tf.lite.Interpreter(model_path=TFLITE_MODEL_PATH)
        interpreter.allocate_tensors()
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()
        print("✅ Model loaded successfully.")
    except Exception as e:
        print(f"Fatal Error: Could not load TFLite model: {e}")
        return

    # --- 2. Get a list of all video files in the input directory ---
    try:
        video_files = [f for f in os.listdir(input_dir) if f.endswith(('.mp4', '.avi', '.mov', '.mkv'))]
        if not video_files:
            print(f"No video files found in '{input_dir}'.")
            return
        print(f"Found {len(video_files)} videos to process.")
    except FileNotFoundError:
        print(f"Fatal Error: Input directory not found at '{input_dir}'")
        return

    # --- 3. Loop through each video file ---
    for video_filename in video_files:
        video_path = os.path.join(input_dir, video_filename)
        
        # Create a unique output directory for this video's results
        video_name_without_ext = os.path.splitext(video_filename)[0]
        output_path_for_video = os.path.join(base_output_dir)
        
        print(f"\n--- Processing video: {video_filename} ---")
        print(f"Output will be saved to: {output_path_for_video}")

        try:
            if not os.path.exists(output_path_for_video):
                os.makedirs(output_path_for_video)
        except Exception as e:
            print(f"Error creating output directory for video: {e}")
            continue # Skip to the next video

        # --- 4. The original video processing logic starts here ---
        try:
            Q = deque(maxlen=128)
            vs = cv2.VideoCapture(video_path)
            if not vs.isOpened():
                print(f"Error: Could not open video file: {video_path}")
                continue # Skip to the next video

            (W, H) = (None, None)
            count = 0
            preds_list = []
        except Exception as e:
            print(f"Error initializing video processing: {e}")
            continue

        while True:
            try:
                (grabbed, frame) = vs.read()
                if not grabbed:
                    break

                if W is None or H is None:
                    (H, W) = frame.shape[:2]

                output = frame.copy()
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = cv2.resize(frame, (128, 128)).astype("float32")
                frame = frame.reshape(128, 128, 3) / 255
                
                input_data = np.expand_dims(frame, axis=0)
                interpreter.set_tensor(input_details[0]['index'], input_data)
                interpreter.invoke()
                preds = interpreter.get_tensor(output_details[0]['index'])[0]
                preds_list.append(preds)

                i = (preds < CONFIDENCE)[0]
                label = 'True' if i else 'False'
                count += 1
            except Exception as e:
                print(f"Error processing frame {count} in {video_filename}: {e}")
                continue

        # Save predictions for the current video to its own CSV
        try:
            preds_df = pd.DataFrame({
                'confidence': [float(p[0]) for p in preds_list]
            })
            csv_path = os.path.join(output_path_for_video, f'{video_name_without_ext}.csv')
            preds_df.to_csv(csv_path, index=False)
            print(f"Predictions for {video_filename} saved to {csv_path}")
        except Exception as e:
            print(f"Error saving predictions to CSV for {video_name_without_ext}: {e}")
        finally:
            vs.release()

    print("\n--- All videos processed. ---")

In [ ]:
process_dir(V_DIR, V_OUTPUT_DIR)
process_dir(NV_DIR, NV_OUTPUT_DIR)